In [1]:
import numpy as np
import scipy.stats as stats
from statsmodels.stats.proportion import proportions_ztest

Поставим себя на место руководителя компании, который хочет прорекламировать свой продукт на некоторой рекламной площадке. Его команда придумала два баннера и стала отслеживать клики по ним, чтобы решить, какой из них оставить. Для этого применем Z-критерий равенства пропорций для независимых выборок, чтобы проверить одну из предполагаемых гипотез:
<br>1. пропорции кликов на баннеры А и В одинаковы
<br>2. на баннер А кликов больше
<br>3. на баннер А кликов меньше

За первые сутки мы собрали следующие данные (здесь 1 - пользователь кликнул на баннер, 0 - нет):

In [2]:
n_A = 300
n_B = 800

banner_A = np.random.randint(0,2, size=(n_A))
banner_B = np.random.randint(0,2, size=(n_B))

banner_A_clicks = banner_A.sum()
banner_B_clicks = banner_B.sum()

print('Баннер А увидело', n_A, 'пользователей, кликнуло на баннер', banner_A_clicks, 'человек')
print('Баннер B увидело', n_B, 'пользователей, кликнуло на баннер', banner_B_clicks, 'человек')

Баннер А увидело 300 пользователей, кликнуло на баннер 152 человек
Баннер B увидело 800 пользователей, кликнуло на баннер 409 человек


Проведем двусторонний тест со следующими гипотезами:
<br>$H_0: \pi_A=\pi_B$, т.е. пропорции кликов по разным баннерам равны
<br>$H_1: \pi_A\neq\pi_B$, т.е. пропорции кликов по разным баннерам НЕ равны

Посмотрим, что дадут нам уже написанные функции из библиотеки statsmodels

In [3]:
count = np.array([banner_A_clicks, banner_B_clicks])
nobs = np.array([n_A, n_B])

z_stat, p_value = proportions_ztest(count, nobs, alternative='two-sided')
print(z_stat, p_value)

-0.13542772833202701 0.8922737020406295


Пробуем самостоятельно написать алгоритм вычисления Z-статистики и ...

In [4]:
p_A = banner_A_clicks/n_A
p_B = banner_B_clicks/n_B
p = (banner_A_clicks + banner_B_clicks) / (n_A + n_B)
Z = (p_A - p_B) / np.sqrt(p*(1-p)/n_A + p*(1-p)/n_B)
Z

-0.13542772833202701

... и p-value.

In [5]:
p_val = stats.norm.cdf(-np.abs(Z)) * 2
p_val

0.8922737020406295

Теперь собирем это в одну функцию

In [6]:
P = {
    'two-sided': stats.norm.cdf(-np.abs(Z)) * 2
}

In [7]:
def my_proportion_ztest(count, nobs, alternative):
    p_1 = count[0]/nobs[0]
    p_2 = count[1]/nobs[1]
    p = (count[0] + count[1]) / (nobs[0] + nobs[1])
    Z = (p_1 - p_2) / np.sqrt(p*(1-p)/nobs[0] + p*(1-p)/nobs[1])
    p_val = P[alternative]
    return Z, p_val

In [8]:
my_proportion_ztest(count, nobs, alternative='two-sided')

(-0.13542772833202701, 0.8922737020406295)

In [9]:
def my_two_ztest(Z , p_val):
    if p_value > 0.05:
        print('Не можем отвергнуть нулевую гипотезу о том, что доли не имеют значимого различия', '\nZ-критерий=',Z,'\np-value = ',p_val)
    else:
        print('Не можем принять нулевую гипотезу о том, что доли не имеют значимого различия','\nZ-критерий=',Z,'\np-value = ',p_val)

In [10]:
my_two_ztest(*my_proportion_ztest(count, nobs, alternative='two-sided'))

Не можем отвергнуть нулевую гипотезу о том, что доли не имеют значимого различия 
Z-критерий= -0.13542772833202701 
p-value =  0.8922737020406295


Теперь проведем  правосторонний тест со следующими гипотезами:
<br>$H_0: \pi_A\leq\pi_B$, т.е. кликов по баннеру А было столько же или меньше, чем кликов по баннеру В
<br>$H_1: \pi_A>\pi_B$, т.е. кликов по баннеру А было значимо больше

Сначала опять посмотрим заранее правильные ответы)))

In [11]:
z_stat, p_value = proportions_ztest(count, nobs, alternative='larger')
print(z_stat, p_value)

-0.13542772833202701 0.5538631489796852


Подсчет Z-статистики никак не отличается, следует изменить только алгоритм подсчета p-value

In [12]:
P = {
    'two-sided': stats.norm.cdf(-np.abs(Z)) * 2,
    'larger': 1 - stats.norm.cdf(Z)
}

Также подкорректируем функцию вывода

In [13]:
def my_right_ztest(Z , p_val):
    if p_value > 0.05:
        print('Не можем отвергнуть нулевую гипотезу о том, что доли кликов на первый баннер меньше или приблизительно равны доли кликов на второй баннер', '\nZ-критерий=',Z,'\np-value = ',p_val)
    else:
        print('Не можем принять нулевую гипотезу о том, что доли кликов на первый баннер меньше или приблизительно равны доли кликов на второй баннер','\nZ-критерий=',Z,'\np-value = ',p_val)

In [14]:
my_right_ztest(*my_proportion_ztest(count, nobs, alternative='larger'))

Не можем отвергнуть нулевую гипотезу о том, что доли кликов на первый баннер меньше или приблизительно равны доли кликов на второй баннер 
Z-критерий= -0.13542772833202701 
p-value =  0.5538631489796852


In [16]:
H0 = {
    'two-sided': ' что доли не имеют значимого различия',
    'larger': ' что доля кликов на первый баннер меньше или приблизительно равна доли кликов на второй баннер'
}

In [17]:
def my_proportion_ztest(count, nobs, alternative):
    p_1 = count[0]/nobs[0]
    p_2 = count[1]/nobs[1]
    p = (count[0] + count[1]) / (nobs[0] + nobs[1])
    Z = (p_1 - p_2) / np.sqrt(p*(1-p)/nobs[0] + p*(1-p)/nobs[1])
    p_val = P[alternative]
    return Z, p_val, alternative 

In [18]:
def my_ztest(Z , p_val, alternative):
    if p_value > 0.05:
        print('Не можем отвергнуть нулевую гипотезу о том,', H0[alternative], '\nZ-критерий=',Z,'\np-value = ',p_val)
    else:
        print('Не можем принять нулевую гипотезу о том,', H0[alternative],'\nZ-критерий=',Z,'\np-value = ',p_val)

In [19]:
my_ztest(*my_proportion_ztest(count, nobs, alternative='larger'))

Не можем отвергнуть нулевую гипотезу о том,  что доля кликов на первый баннер меньше или приблизительно равна доли кликов на второй баннер 
Z-критерий= -0.13542772833202701 
p-value =  0.5538631489796852


Теперь проведем левосторонний тест с обратными гипотезами:
<br>$H_0: \pi_A\geq\pi_B$, т.е. кликов по баннеру А было столько же или больше, чем кликов по баннеру В
<br>$H_1: \pi_A<\pi_B$, т.е. кликов по баннеру А было значимо меньше

Сравнивать будем с этим...

In [20]:
z_stat, p_value = proportions_ztest(count, nobs, alternative='smaller')
print(z_stat, p_value)

-0.13542772833202701 0.44613685102031475


Нужно предусмотреть этот вариант в функции вычисления p-value и функции, отображающей конечные выводы

In [21]:
P = {
    'two-sided': stats.norm.cdf(-np.abs(Z)) * 2,
    'larger': 1 - stats.norm.cdf(Z),
    'smaller': stats.norm.cdf(Z)
}

In [22]:
H0 = {
    'two-sided': ' что доли не имеют значимого различия',
    'larger': ' что доля кликов на первый баннер меньше или приблизительно равна доли кликов на второй баннер',
    'smaller': ' что доля кликов на первый баннер больше или приблизительно равна доли кликов на второй баннер'
}

Теперь остается только запустить нашу функцию

In [23]:
my_ztest(*my_proportion_ztest(count, nobs, alternative='smaller'))

Не можем отвергнуть нулевую гипотезу о том,  что доля кликов на первый баннер больше или приблизительно равна доли кликов на второй баннер 
Z-критерий= -0.13542772833202701 
p-value =  0.44613685102031475
